# Minimalist Algebras Demo: MG+2.3

This Notebook will take you through some of the features of the Python code I wrote to implement my notion of Generalised Minimalist Algebras.

Requirements for this demo: `nltk` should probably do it.

These implement only the structure-building component of an MG. From the two-step perspective, these are just for the second step. (To incorporate features, there need to be rules mapping an operation in a given feature state to the algebra operation. These are implemented for John Torr's MGBank grammar, but I won't talk about those today. You can find them in `minimalist_parser.convert_mgbank`.)

This code is very object-oriented with a lot of inheritance and default behaviour.

We will start with the `Algebra` class.

In [ ]:
# bunch o'imports
from minimalist_parser.algebras.string_algebra import BareTreeStringAlgebra
from minimalist_parser.algebras.algebra import AlgebraOp, AlgebraTerm
from minimalist_parser.minimalism.minimalist_algebra_synchronous import MinimalistAlgebraSynchronous, MinimalistFunctionSynchronous, SynchronousTerm, InnerAlgebraInstructions
from minimalist_parser.algebras.hm_algebra import HMAlgebra
from minimalist_parser.minimalism.prepare_packages.prepare_packages_bare_trees import PreparePackagesBareTrees
from minimalist_parser.minimalism.prepare_packages.prepare_packages_hm import PreparePackagesHM

We start by initialising an algebra over strings with operation names < and >. The terms over this algebra are thus Bare Trees a la Stabler 1997.

`BareTreeStringAlgebra` is a subclass of `HMAlgebra`, which I designed for head movement.

In [ ]:
string_algebra = BareTreeStringAlgebra()

help(string_algebra)

Algebra constants are zeroary operations. They have a name and a zeroary function, which is an element of the domain: here, a string.

(With an algebra this simple, it's hard to see why the name and function should be separate, but for, say, a tree algebra or graph algebra, it's more clear.)

In [ ]:
the = AlgebraOp(name="the", function="the")

# The algebra also has a default constant maker
puppy = string_algebra.constant_maker("puppy")
snuggled = string_algebra.constant_maker("snuggled")

# look at the vocabulary
vocab = [the, puppy, snuggled]
for algebra_op in vocab:
    print("\nname:", algebra_op.name)
    print("function:", algebra_op.function)


Binary operations < and > can be looked up in `string_algebra.ops` or they can be created. 

For consistency across Head Movement Algebras, < is in `string_algebra.ops['concat_right']` and > is in `string_algebra.ops['concat_left']`. Notice both use the string_algebra.concat_right method, since Bare Trees are WYSIWYG when it comes to word order.

In [ ]:
print("name:", string_algebra.ops['concat_right'].name)
print("function:", string_algebra.ops['concat_right'].function)
print("\nname:", string_algebra.ops['concat_left'].name)
print("function:", string_algebra.ops['concat_left'].function)

You can always just create an `AlgebraOp` as well, if you want.

In [ ]:
new_right = AlgebraOp("<", string_algebra.concat_right)
print(new_right == string_algebra.ops['concat_right'])

### Algebra Terms

An term over an algebra is a tree in which the nodes are labelled with operations of that algebra, and the number of children of the node matches the arity of the function of the operation.

This means the leaves are constants, and the internal nodes are operations. 

An `AlgebraTerm` has a `parent`, which is an `AlgebraOp`, and, optionally, a list of `children`, which are `AlgebraTerm`s.

In [ ]:
# for readability, let's give < and > names
# Type: AlgebraOp
head_left = string_algebra.ops['concat_right']
head_right = string_algebra.ops['concat_left']

# we also really want our vocabulary to be AlgebraTerms. We can turn them all into AlgebraTerms, but instead let's make them with the convenience function make_leaf
# Type: AlgebraTerm
the = string_algebra.make_leaf("the")
puppy = string_algebra.make_leaf("puppy")
snuggled = string_algebra.make_leaf("snuggled")

the_puppy = AlgebraTerm(head_left, [the, puppy])

print(the_puppy)

`AlgebraTerm`s can be exported to `nltk.Tree`s, which allows us to visualise them.

In [ ]:
the_puppy.to_nltk_tree().draw()

In [ ]:
the_puppy_snuggled = AlgebraTerm(head_right, [the_puppy, snuggled])
the_puppy_snuggled.to_nltk_tree().draw()

## Minimalist Algebras

A Minimalist Algebra is also an algebra, but it's quite different from a string-building algebra. A Minimalist Algebra essentially handles all the Move-related work. Structure-building is delegated to its "inner algebra", which can be, for instance, this `string_algebra`.

The domain of a Minimalist Algebra is `Expression`s, which contain an `inner_term` (a term of the inner algebra), `Movers`, and have an `mg_type` with things like +/- lexical and +/- conjunction.

`Movers` implements a partial function from slot names (such as `'-wh'` or `'Abar'`) to inner terms. 

Today we'll just look at the `MinimalistAlgebraSynchronous` subclass, so I can show you how to build a synchronous grammar over multiple inner algebras.

In [ ]:
# a MinimalistAlgebraSynchronous requires a list of inner algebras.
minimalist_algebra = MinimalistAlgebraSynchronous([string_algebra])

help(minimalist_algebra)

## Adding your own inner algebra

To do this, implement an `Algebra`. The `HMAlgebra` class has a bunch of default functions, so if you want an algebra with head movement, you may find this convenient. The `concat_right` etc methods by default just return `arg[0] + arg[1]` (or vice versa), so if you have a class you want to build, you may be able to get away with just initialising an `HMAlgebra` with `domain_type` specified.

In the original Chomsky 1995, he builds multisets of multisets. Python won't let you do that, so I implemented `FakeSet`, which inherits from `list`, but ignores order.

With this, you don't need to write a new class of `HMAlgebra`, just initialise one with `domain_type=FakeSet`.

In [ ]:
from minimalist_parser.algebras.algebra_objects.fake_set import FakeSet

set_algebra = HMAlgebra("set algebra", FakeSet)

`HMAlgebra`s don't by default have a constant maker. We need to write our own, since a constant (lexical item) should already be a `FakeSet`, not just a string.

In [ ]:
def make_fake_set_constant(word):
    """
    make an AlgebraOp with the given word as the content of a unary FakeSet.
    With this we can use the synchronous algebra make_leaf function for a shortcut to a term leaf.
    """
    return AlgebraOp(word, FakeSet([word]))

set_algebra.add_constant_maker(make_fake_set_constant)

In [ ]:
# this is just the built-in concat_right method of HMAlgebras, which just uses + on the two arguments.
print(set_algebra.concat_right([FakeSet([7,8,1]), FakeSet([3, 2])]).spellout())

In [ ]:
# a term
# set_algebra.ops["concat_right"] is an AlgebraOp with name 'concat_right' and function set_algebra.concat_right.
t = AlgebraTerm(set_algebra.ops["concat_right"], [set_algebra.make_leaf("MG+"), AlgebraTerm(set_algebra.ops["concat_right"], [set_algebra.make_leaf("hi"), set_algebra.make_leaf("there")])])
t.to_nltk_tree().draw()
t.evaluate()


In [ ]:
# ignore these for now, they're just to make our lives easier later.
string_prepare = PreparePackagesBareTrees()
set_prepare = PreparePackagesHM("set prepare", set_algebra)


In [ ]:
# ignore the prepare_packages part. They're not necessary, but they save us re-making examples later.
minimalist_algebra = MinimalistAlgebraSynchronous([string_algebra, set_algebra], prepare_packages=[string_prepare, set_prepare])

In [ ]:
# The operations of an algebra don't have to be stored in algebra.ops.
# by default, a minimalist algebra doesn't have any stored here. The operations are just built using MinimalistFunctionSynchronous.

# However, since all the inner algebras are HMAlgebras, there's a method for just adding all possible minimalist operations, right and left, given the movers slots.
minimalist_algebra.add_default_operations()

In [ ]:
for op in minimalist_algebra.ops:
    print(op)

In [ ]:
# for readability, let's get some operations
merge_right = minimalist_algebra.ops["Merge1_right"]
merge_left = minimalist_algebra.ops["Merge1_left"]
merge_A = minimalist_algebra.ops["Merge2_A"]
move_A = minimalist_algebra.ops["Move1_left_A"]
merge_Abar = minimalist_algebra.ops["Merge2_ABar"]
move_Abar = minimalist_algebra.ops["Move1_left_ABar"]
move_A_Abar = minimalist_algebra.ops["Move2_A_ABar"]

In [ ]:
# and some constants
the = minimalist_algebra.make_leaf("the")
puppy = minimalist_algebra.make_leaf("puppy")
snuggled = minimalist_algebra.make_leaf("snuggled")


If you don't want to use the default constant maker, you can pass `make_leaf` a dict from inner algebra to `InnerAlgebraInstructions`.

For example, if you need silent heads:

In [ ]:
# we can usually just make a silent thing with the constructor of the inner domain.
# for example, str() makes "" and FakeSet() makes {}
# using this, inner_algebra.empty_leaf_operation should work for most Algebras
silent_inners = {inner_algebra: InnerAlgebraInstructions(algebra_op=inner_algebra.empty_leaf_operation) for inner_algebra in minimalist_algebra.inner_algebras}
past = minimalist_algebra.make_leaf("[past]", silent_inners)

# see the silent heads:
print(set_algebra, past.parent, past.spellout(set_algebra))
print(string_algebra, past.parent, past.spellout(string_algebra))


In [ ]:
# A term
t=SynchronousTerm(move_A, [SynchronousTerm(merge_right, [past, SynchronousTerm(merge_A, [snuggled, SynchronousTerm(merge_right, [the, puppy])])])])
t.to_nltk_tree().draw()

# See the inner terms
t.interp(set_algebra).inner_term.to_nltk_tree().draw()
t.interp(string_algebra).inner_term.to_nltk_tree().draw()



`spellout` is a shortcut for `t.interp(algebra).inner_term.evaluate()`. If the `domain_type` also has a `spellout` method, it also applies that, so e.g. a tree could spellout to its string yield or a `(string, triple, "")` could spell out to `"string triple"`.


In [ ]:
print(t.spellout(set_algebra))
print(t.spellout(string_algebra))

In [ ]:
# Move2
who = minimalist_algebra.make_leaf("who")
q = minimalist_algebra.make_leaf("[Q]", silent_inners)


t2 = SynchronousTerm(move_Abar, [SynchronousTerm(merge_right, [q, SynchronousTerm(move_A_Abar, [SynchronousTerm(merge_right, [past, SynchronousTerm(merge_A, [snuggled, who])])])])])

# Minimalist term
t2.to_nltk_tree().draw()

# inner terms
t2.interp(set_algebra).inner_term.to_nltk_tree().draw()
t2.interp(string_algebra).inner_term.to_nltk_tree().draw()

# spellout
print(t2.spellout(set_algebra))
print(t2.spellout(string_algebra))



## Prepare Packages

So far, we've had no Head Movement and moved items are not marked with traces. These kinds of things require* tree homomorphisms on the inner algebra terms.

These are called `PreparePackages`, and are paired with the `Algebra`s. `PreparePackagesHM` have methods to extract and combine heads.

`*` "require" is too strong a word. They can be built into the algebras if you want, but this misses generalisations, and they'll look different on the inner terms.


In [ ]:
# for the string algebra, these are special since we need to follow with <, > arrows to find the head.
# by default, we just go down the left branches.
string_prepare = PreparePackagesBareTrees()
set_prepare = PreparePackagesHM("set prepare", set_algebra)


In [ ]:
# e.g. the method extract_head returns a pair of the tree without its head and the head
snuggled_puppy = SynchronousTerm(merge_left, [snuggled, puppy])

# sets
remainder, head = set_prepare.extract_head(snuggled_puppy.interp(set_algebra).inner_term)
remainder.to_nltk_tree().draw()
head.to_nltk_tree().draw()

# removing the head in the string algebra leaves a trace t
remainder, head = string_prepare.extract_head(snuggled_puppy.interp(string_algebra).inner_term)
remainder.to_nltk_tree().draw()
head.to_nltk_tree().draw()

A prepare package is a pair of tree homomorphisms that change, finitely, the functor term and the other term. ("other" = mover, or selectee, or modified)

In an HM algebra, these are just for head movement. As part of a minimalist operation, the output of the prepare package will then be combined with an operation of the inner algebra, such as `concat_right`.

The standard Prepare Packages for head movement algebras are:

* `suffix`: functor, other -> (functor with head = h_functor + other_functor , other without its head)
* `prefix`: functor, other -> (functor with head = other_functor h_functor , other without its head)
* `excorporation`: functor, other -> (other head , concat_right(functor head, other without its head)
* `hm_atb`: ONLY if functor head == other head:
  * functor, other -> (functor head, concat_right(functor without its head, other without its head)

A `MiniimalistFunction` is a type of `AlgebraOp` with a very complex constructor, taking things like the inner algebra operation, the prepare package, if any, and the slot(s) for movement.

For synchronous algebras, a `MinimalistFunctionSynchronous`, the inner-algebra-specific information is gathered in a dict from algebra to `InnerAlgebraInstructions`

In [ ]:
help(InnerAlgebraInstructions)

In [ ]:
# since both of our algebras are HMAlgebras with PreparePackagesHM prepare packages, we can make in inner_ops dict very easily.
# for example, we can create a new minimalist operation with head-raising to prefix position.
prefix = {a: InnerAlgebraInstructions(op_name="concat_right", prepare="prefix") for a in minimalist_algebra.inner_algebras}

merge_right_prefix = MinimalistFunctionSynchronous(minimalist_algebra, minimalist_algebra.merge1, inner_ops=prefix, name="Merge1_right_prefix")

# we can add it to the algebra if we want.
minimalist_algebra.add_op(merge_right_prefix)

In [ ]:
ed = minimalist_algebra.make_leaf("-ed")
walk = minimalist_algebra.make_leaf("walk")

walked = SynchronousTerm(merge_right_prefix, [ed, walk])
walked.to_nltk_tree().draw()

# shortcut for walked.interp(string_algebra).inner_term.to_nltk_tree().draw()
walked.view_inner_term(string_algebra)
walked.spellout(string_algebra)

## Usage: we can make parse items without actually having to parse a sentence

In [ ]:
from minimalist_parser.examples import tree_atb, cat_tried_sleep, tree_atb_hm, mg, triple_alg, am_alg
from minimalist_parser.convert_mgbank.term2actions import add_interval_algebra
from minimalist_parser.minimalism.prepare_packages.addressed_triple_prepare_package import \
    HMAddressedTriplesPreparePackages
from minimalist_parser.algebras.hm_triple_algebra import HMTripleAlgebra
from minimalist_parser.minimalism.prepare_packages.interval_prepare_package import IntervalPairPrepare
from minimalist_parser.algebras.hm_interval_pair_algebra import HMIntervalPairsAlgebra

In [ ]:
terms = tree_atb, cat_tried_sleep, tree_atb_hm
for term in terms:
    print(term.spellout(triple_alg))
    term.to_nltk_tree().draw()
    term.view_inner_term(triple_alg)
    

In [ ]:
# make required algebras and add to MG

# IntervalPairs implement pairs of intervals a la Milos's paper on parsing complexity with head movement
# (but my version)
# (head interval, rest of the phrase interval) + typing to tell you what operations can apply (must_hm and lexical)
example_interval_algebra = HMIntervalPairsAlgebra()
example_interval_prepare = IntervalPairPrepare()
example_interval_algebra.add_constant_maker()

# interpret into MG tree addresses, to track where words in the sentence came from (including deletion due to ATB movement)
example_address_algebra = HMTripleAlgebra("addresses algebra", component_type=list, addresses=True)
example_address_prepare = HMAddressedTriplesPreparePackages()

mg.inner_algebras[example_address_algebra] = example_address_prepare
mg.inner_algebras[example_interval_algebra] = example_interval_prepare

# use the addresses as output to add an interpretation over intervals in the string
for term in [tree_atb, cat_tried_sleep, tree_atb_hm]:
    add_interval_algebra(term, mg, triple_alg, example_address_algebra, example_address_prepare,
                         example_interval_algebra, example_interval_prepare, False)


In [ ]:
print(tree_atb.spellout(example_address_algebra))
print(tree_atb.spellout(triple_alg))
print(tree_atb.spellout(example_interval_algebra))
tree_atb.to_nltk_tree().draw()

In [ ]:
# we can see the string indices in the interval algebra term
tree_atb.view_inner_term(example_interval_algebra)

In [ ]:
for alg in mg.inner_algebras:
    print()
    print(alg)
    print(tree_atb.spellout(alg))

### Bonus: Graph algebra

These examples have a graph algebra interpretation as well. I don't have any built-in visualiser, but you can export them to GraphViz.

In [ ]:
with open("data/processed/graphs/mg_plus.dot", 'w') as f:
    for term in terms:
        g = term.spellout(am_alg)
    
        dot = g.to_graphviz()
        f.write(dot)
        f.write("\n\n")

In [ ]:
for term in terms:
    term.view_inner_term(am_alg)